In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [ ]:
import numpy as np
import pandas as pd
from datasets import Dataset
from collections import Counter
from datasets import concatenate_datasets

In [ ]:
from datasets import load_dataset
raid = load_dataset("liamdugan/raid")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.8G [00:00<?, ?B/s]

extra.csv:   0%|          | 0.00/3.71G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating extra split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
print(raid)

train = raid["train"]

print(train)
models = train["model"]
attacks = train["attack"]

unique_models = set(models)
unique_attacks = set(attacks)

DatasetDict({
    train: Dataset({
        features: ['id', 'adv_source_id', 'source_id', 'model', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'generation'],
        num_rows: 5615820
    })
    extra: Dataset({
        features: ['id', 'adv_source_id', 'source_id', 'model', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'generation'],
        num_rows: 2039100
    })
})
Dataset({
    features: ['id', 'adv_source_id', 'source_id', 'model', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'generation'],
    num_rows: 5615820
})


In [ ]:
print("Список всех моделей в train датасете:")
for model in unique_models:
    print(model)

print("Список всех атак в train датасете:")
for attack in unique_attacks:
    print(attack)

Список всех моделей в train датасете:
mpt
cohere-chat
gpt3
chatgpt
mistral
mpt-chat
gpt2
llama-chat
gpt4
human
mistral-chat
cohere
Список всех атак в train датасете:
whitespace
none
paraphrase
upper_lower
homoglyph
article_deletion
synonym
perplexity_misspelling
insert_paragraphs
number
alternative_spelling
zero_width_space


In [ ]:
wiki_dataset = train.filter(lambda example: example["domain"] == "reddit")

print(f"Число записей в отфильтрованном датасете: {len(wiki_dataset)}")
print(wiki_dataset[0])

Filter:   0%|          | 0/5615820 [00:00<?, ? examples/s]

Число записей в отфильтрованном датасете: 747180
{'id': '3cabbb8e-2a0c-410f-8fb8-15305ff49230', 'adv_source_id': '3cabbb8e-2a0c-410f-8fb8-15305ff49230', 'source_id': '3cabbb8e-2a0c-410f-8fb8-15305ff49230', 'model': 'human', 'decoding': None, 'repetition_penalty': None, 'attack': 'none', 'domain': 'reddit', 'title': '350hrs in and I still have no idea what I’m doing...', 'prompt': None, 'generation': 'So, I’ve been playing the game on and off for the past 3 years and I still feel like a noob. Something that I really struggle with is combat. I just spent my entire weekend putting together a $185 million fer-de-lance build. The moment I entered the battlefield I was destroyed. I thought the FDL was meant to have insane shielding and maneuverability but my shields were gone in 30 seconds and I was outmaneuvered by a python. My hull was destroyed in about 10 seconds. Even with 4 seeking missiles I only dealt 30% to the python. WTF. Can anyone tell me why this happens and how to get better i

In [ ]:
human_data = wiki_dataset.filter(lambda x: x["model"] == "human")
non_human_data = wiki_dataset.filter(lambda x: x["model"] != "human")

Filter:   0%|          | 0/747180 [00:00<?, ? examples/s]

Filter:   0%|          | 0/747180 [00:00<?, ? examples/s]

In [ ]:
human_attacks = list(set(human_data["attack"]))
total_human_samples = 18000
distance_samples_total = 3000
distance_split = distance_samples_total // len(human_attacks)
samples_per_human_attack = (total_human_samples - distance_samples_total) // len(human_attacks)

distance_parts = []
balanced_human_train = []
balanced_human_test = []

for attack in human_attacks:
    attack_data = human_data.filter(lambda x: x["attack"] == attack).shuffle(seed=42)

    distance_part = attack_data.select(range(distance_split))
    distance_parts.append(distance_part)

    remaining = attack_data.select(range(distance_split, len(attack_data)))
    n_samples = min(samples_per_human_attack, len(remaining))

    test_size = int(0.2 * n_samples)
    train_size = n_samples - test_size

    test_part = remaining.select(range(test_size))
    train_part = remaining.select(range(test_size, test_size + train_size))

    balanced_human_test.append(test_part)
    balanced_human_train.append(train_part)

distance_train = concatenate_datasets(distance_parts)
human_test = concatenate_datasets(balanced_human_test)
human_train = concatenate_datasets(balanced_human_train)

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

Filter:   0%|          | 0/21348 [00:00<?, ? examples/s]

In [ ]:
non_human_models = list(set(non_human_data["model"]))
non_human_attacks = list(set(non_human_data["attack"]))

total_non_human_samples = len(human_test) + len(human_train)
samples_per_combo = total_non_human_samples // (len(non_human_models) * len(non_human_attacks))

balanced_non_human_train = []
balanced_non_human_test = []

for model in non_human_models:
    model_data = non_human_data.filter(lambda x: x["model"] == model)
    for attack in non_human_attacks:
        combo_data = model_data.filter(lambda x: x["attack"] == attack).shuffle(seed=42)

        n_samples = min(samples_per_combo, len(combo_data))
        test_size = int(0.2 * n_samples)
        train_size = n_samples - test_size

        test_part = combo_data.select(range(test_size))
        train_part = combo_data.select(range(test_size, test_size + train_size))

        balanced_non_human_test.append(test_part)
        balanced_non_human_train.append(train_part)

test_models = concatenate_datasets(balanced_non_human_test)
train_models = concatenate_datasets(balanced_non_human_train)

Filter:   0%|          | 0/725832 [00:00<?, ? examples/s]

Filter:   0%|          | 0/85392 [00:00<?, ? examples/s]

Filter:   0%|          | 0/85392 [00:00<?, ? examples/s]

Filter:   0%|          | 0/85392 [00:00<?, ? examples/s]

Filter:   0%|          | 0/85392 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/42696 [00:00<?, ? examples/s]

Filter:   0%|          | 0/42696 [00:00<?, ? examples/s]

Filter:   0%|          | 0/42696 [00:00<?, ? examples/s]

Filter:   0%|          | 0/42696 [00:00<?, ? examples/s]

Filter:   0%|          | 0/42696 [00:00<?, ? examples/s]

In [ ]:
test = concatenate_datasets([human_test] + balanced_non_human_test)
train = concatenate_datasets([human_train] + balanced_non_human_train)

print("Общий размер датасета:", len(test))
print("Общий размер датасета:", len(train))
print("Общий размер датасета:", len(distance_train))

Общий размер датасета: 5904
Общий размер датасета: 24012
Общий размер датасета: 3000


In [ ]:
distance_train_df = distance_train.to_pandas()
final_train_df = train.to_pandas()
final_test_df = test.to_pandas()

distance_train_df = distance_train_df[["model", "attack", "generation"]]
final_train_df = final_train_df[["model", "attack", "generation"]]
final_test_df = final_test_df[["model", "attack", "generation"]]

distance_train_df.to_csv("distance_train.csv", index=False)
final_train_df.to_csv("train.csv", index=False)
final_test_df.to_csv("test.csv", index=False)